## Match BACI trade flows to ecoinvent processes

In [ ]:
import pandas as pd
import numpy as np
import os
import sys
# your path to pylacio
sys.path.append('/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/')
import pylcaio
sys.path.append(os.path.realpath('../'))
import match_BACI_price_data_to_ecoinvent
import make_price_df


First we will match Baci trade flows to ecoinvent processes using the two mapping dictionaries (region and commodity). 

In [ ]:
# For this we will need the meta data on ecoinvent processes which we get from the output of pylcaio
your_path_to_hybrid_db = '/home/jakobs/Documents/IndEcol/OASES/pylcaio/src/Databases/ecoinvent3.5_exiobase3_2012/hybrid_system_STAM.pickle'
hybrid_db = pylcaio.Analysis(your_path_to_hybrid_db)
PRO = pd.DataFrame.from_dict(hybrid_db.PRO_f)

#Let's see what's inside
PRO


The matching may take a while (up to a few hours depending on your machine) You can check the Ray dashboard to see your cpu load. Every 100 processes you it will print its progress.

In [ ]:
USD_per_EURO = 1.2848  # 2012 EUROSTAT annual average 
# (note that EUROstat gives the USD per EURO rate, the dollar price will be divided by this number 
# to obtain the price in EURO's)
your_path_to_baci_data = '/home/jakobs/Documents/IndEcol/Data/BACI/BACI_HS12_V202001/BACI_HS12_Y2012_V202001.csv'
directory_to_save_price_data = 'directory_to_save_price_data_for_ei_processes/'
mapping_data_dir = 'mapping_data/'

# For some reason Ray has a problem with the system paths and does not 
# find the match_BACI_price_data_to_ecoinvent module so change paths now and then later change back
# !Note that this has an impact on the paths above if relative paths were given!
cwd = os.getcwd()
os.chdir('../')
match_BACI_price_data_to_ecoinvent.Match_BACI_data_to_ecoinvent(PRO=PRO, path_to_BACI_data=your_path_to_baci_data,
                                      outputDir=directory_to_save_price_data,
                                      USD_EURO_exr=USD_per_EURO, mapping_data_dir=mapping_data_dir)
os.chdir(cwd)

### Creating a dataframe with all price distributions. 

This will read in the BACI price distributions generated above. Price distributions of processes for which BACI data is not available will be modelled as a lognormal distribution with a mean at the ecoinvent price and a shape paramter based on proxydata. There are deafult parameters from proxy data. See DOI: for the proxy data used.

In [ ]:
price_data_file_name = 'Price_data_ei_processes_BACI_2012.ftr'
price_data_output_dir = '../directory_to_save_price_data_feather_file/'
make_price_df.make_price_df(PRO=PRO,
                            price_data_dir=directory_to_save_price_data,
                            hybridized_processes=hybrid_db.hybridized_processes,
                            outputFileName=price_data_file_name,
                            outputDir=price_data_output_dir)

Now the above feather file with price data can be used. See jupyter notebook "Using hybrid database with BACI price data" on how to use it in a MC analysis of uncertainty of a hybrid LCA carbon footprint due to price variance. 